In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from statsmodels.tsa.stattools import adfuller
import seaborn as sns
from pmdarima.arima import auto_arima
from sklearn.metrics import r2_score
import math
from sklearn.metrics import mean_squared_error

%matplotlib inline 
mpl.rcParams['figure.figsize']=(10,8)
mpl.rcParams['axes.grid']=False
    

In [ ]:
df=pd.read_csv('calls_data.csv')

In [ ]:
array=[]
mean_err=[]
for i in range(1,301):
      array=df.loc[df['user'] == i]
      array=array.drop(['duration','user'],axis=1)
      array.rename(columns = {'time_normalized':i}, inplace = True)
      array=array.values.tolist()
      df_id=pd.DataFrame(array)
      print("-------------------------------------------user "+str(i)+"-------------------------------------------")
      test_res = adfuller(df_id)
      def adfuller_test(sales):
          result=adfuller(sales)
          labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
          for value,label in zip(result,labels):
              print(label+' : '+str(value) )
          if result[1] <= 0.05:
              print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
          else:
              print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")
      k=len(df_id)
      train = df_id[:int(0.8*k)]
      test = df_id[int(0.8*k):]
      arima_model =  auto_arima(train,start_p=0, d=1, start_q=0, 
                            max_p=10, max_d=10, max_q=10, start_P=0, 
                            D=1, start_Q=0, max_P=5, max_D=5,
                            max_Q=5, m=2,seasonal=True, 
                            error_action='warn',trace = True,
                            supress_warnings=True,stepwise = True,
                            random_state=20,n_fits = 50)
      prediction = pd.DataFrame(arima_model.predict(n_periods = len(test)),index=test.index)
      #print(r2_score(test, prediction))
      test.rename(columns = {0:'True_data'}, inplace = True)
      prediction.rename(columns = {0:'Pred_data'}, inplace = True)
      print("---------------------------------mean_squared_error----------------------------------")
      print(math.sqrt(mean_squared_error(test,prediction)))
      mean_err.append(math.sqrt(mean_squared_error(test,prediction)))
      r = pd.concat([prediction,test], axis=1)
      r['True_data'].plot(figsize=(12,8))
      r['Pred_data'].plot(figsize=(12,8))
      plt.xlabel('next access')
      plt.ylabel('Time normalized')
      plt.show()
      r.to_csv('user_'+str(i)+'.csv',
                index = None)